<a href="https://colab.research.google.com/github/Prast667/Prit/blob/main/pp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import zipfile
from google.colab import drive

# --- KODE SETUP AWAL ---
print("Menghubungkan Google Drive...")
drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/Sinta 2.zip'
extract_dir = '/content/data_sampah/'
FOLDER_INDUK = 'Sinta 2' # NAMA FOLDER INDUK YANG BARU DITEMUKAN

# Membuat direktori ekstraksi jika belum ada
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

print(f"Mengekstrak {zip_path} ke {extract_dir}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Ekstraksi selesai!")

# --- PENYESUAIAN PATH FINAL ---

# Path TRAIN dan TEST sekarang harus melalui FOLDER_INDUK
TRAIN_DIR = os.path.join(extract_dir, FOLDER_INDUK, 'Train')
TEST_DIR = os.path.join(extract_dir, FOLDER_INDUK, 'Test')

# Verifikasi Path
print(f"\nVerifikasi Path TRAIN: {TRAIN_DIR}")
print(f"Isi dari TRAIN_DIR: {os.listdir(TRAIN_DIR)}")
# Output harusnya ['Organik', 'Anorganik'] atau nama kelas Anda yang sebenarnya

Menghubungkan Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mengekstrak /content/drive/MyDrive/Sinta 2.zip ke /content/data_sampah/...
Ekstraksi selesai!

Verifikasi Path TRAIN: /content/data_sampah/Sinta 2/Train
Isi dari TRAIN_DIR: ['Anorganik', 'Organik']


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# =======================================================
# IMAGE DATA GENERATOR UNTUK DATA TRAINING (AUGMENTASI)
# =======================================================
train_datagen = ImageDataGenerator(
    rescale=1./255,                 # Normalisasi Wajib
    rotation_range=15,              # Rotasi antara -15 dan 15 derajat
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,           # Rotate horizontal
    vertical_flip=True,             # Rotate vertical
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

# =======================================================
# IMAGE DATA GENERATOR UNTUK DATA TESTING (HANYA NORMALISASI)
# =======================================================
test_datagen = ImageDataGenerator(rescale=1./255)


# =======================================================
# PEMUATAN DATA (GENERATOR)
# =======================================================
try:
    # Training Generator
    train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    # Test/Validation Generator
    test_generator = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    print("\n✅ Data Generator berhasil dibuat!")
    print(f"Total gambar train: {train_generator.samples}")
    print(f"Total gambar test: {test_generator.samples}")

except Exception as e:
    # Jika masih gagal, mungkin nama folder kelas (Organik/Anorganik) tidak cocok
    print(f"\n❌ GAGAL MEMUAT GENERATOR. Periksa kembali struktur folder di: {TRAIN_DIR}")
    print(f"Error: {e}")

Found 1767 images belonging to 2 classes.
Found 428 images belonging to 2 classes.

✅ Data Generator berhasil dibuat!
Total gambar train: 1767
Total gambar test: 428


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Tentukan ukuran input yang umum
IMG_SIZE = (224, 224)

def build_transfer_model(base_model_func, input_shape, model_name, learning_rate=1e-4):
    """Membangun model Transfer Learning."""

    # Memuat model pra-latih (ImageNet weights) tanpa lapisan Fully Connected
    base_model = base_model_func(weights='imagenet',
                                 include_top=False,
                                 input_shape=input_shape)

    # Membekukan lapisan dasar (Feature Extraction)
    base_model.trainable = False

    # Menambahkan lapisan klasifikasi kustom di atas
    x = base_model.output
    x = GlobalAveragePooling2D(name='global_average_pooling')(x)
    x = Dense(256, activation='relu', name='custom_dense_1')(x)
    predictions = Dense(1, activation='sigmoid', name='output_layer')(x) # Output biner

    # Menggabungkan model dasar dan lapisan baru
    model = Model(inputs=base_model.input, outputs=predictions, name=model_name)

    # Mengkompilasi model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Mendefinisikan Callbacks untuk semua model
# Early Stopping: Menghentikan training jika validasi akurasi tidak membaik
early_stopping = EarlyStopping(monitor='val_accuracy',
                               patience=5,
                               restore_best_weights=True)

# Model Checkpoint: Menyimpan model terbaik
# Ganti nama file ini saat training model yang berbeda
# checkpoint = ModelCheckpoint('best_model_NAME.h5', monitor='val_accuracy', save_best_only=True)

In [15]:
from tensorflow.keras.applications import ResNet50

print("\n--- Melatih ResNet50 ---")
resnet_model = build_transfer_model(ResNet50, IMG_SIZE + (3,), 'ResNet50_Transfer')

# Tampilkan ringkasan model
# resnet_model.summary()

resnet_history = resnet_model.fit(
    train_generator,
    epochs=50, # Jumlah epochs bisa disesuaikan
    validation_data=test_generator,
    callbacks=[early_stopping], # Tambahkan callback
    verbose=1
)


--- Melatih ResNet50 ---
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
 5/56 ━━━━━━━━━━━━━━━━━━━━ 4:16 5s/step - accuracy: 0.4920 - loss: 0.7660

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

print("\n--- Melatih EfficientNetB0 ---")
efficientnet_model = build_transfer_model(EfficientNetB0, IMG_SIZE + (3,), 'EfficientNetB0_Transfer')

efficientnet_history = efficientnet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
from tensorflow.keras.applications import DenseNet121

print("\n--- Melatih DenseNet121 ---")
densenet_model = build_transfer_model(DenseNet121, IMG_SIZE + (3,), 'DenseNet121_Transfer')

densenet_history = densenet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
from tensorflow.keras.applications import MobileNetV2

print("\n--- Melatih MobileNetV2 ---")
mobilenet_model = build_transfer_model(MobileNetV2, IMG_SIZE + (3,), 'MobileNetV2_Transfer')

mobilenet_history = mobilenet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
from tensorflow.keras.applications import VGG16

# VGG16 cenderung lebih lambat dan membutuhkan banyak memori
print("\n--- Melatih VGG16 ---")
vgg16_model = build_transfer_model(VGG16, IMG_SIZE + (3,), 'VGG16_Transfer')

vgg16_history = vgg16_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
from tensorflow.keras.applications import NASNetMobile

print("\n--- Melatih NASNetMobile ---")
nasnet_model = build_transfer_model(NASNetMobile, IMG_SIZE + (3,), 'NASNetMobile_Transfer')

nasnet_history = nasnet_model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
import pandas as pd

results = {
    'Model': ['ResNet50', 'EfficientNetB0', 'DenseNet121', 'MobileNetV2', 'NASNetMobile', 'VGG16'],
    'Validation Accuracy': [
        max(resnet_history.history['val_accuracy']),
        max(efficientnet_history.history['val_accuracy']),
        max(densenet_history.history['val_accuracy']),
        max(mobilenet_history.history['val_accuracy']),
        max(nasnet_history.history['val_accuracy']),
        max(vgg16_history.history['val_accuracy'])
    ],
    'Training Loss (Last Epoch)': [
        resnet_history.history['loss'][-1],
        efficientnet_history.history['loss'][-1],
        densenet_history.history['loss'][-1],
        mobilenet_history.history['loss'][-1],
        nasnet_history.history['loss'][-1],
        vgg16_history.history['loss'][-1]
    ]
}

df_results = pd.DataFrame(results)
print("\n--- Hasil Perbandingan Model ---")
print(df_results.sort_values(by='Validation Accuracy', ascending=False))

In [ ]:
import matplotlib.pyplot as plt

# Kumpulkan semua history object ke dalam satu dictionary
# GANTI DENGAN NAMA HISTORY OBJECT ANDA
all_histories = {
    "ResNet50": resnet_history,
    "EfficientNetB0": efficientnet_history,
    "DenseNet121": densenet_history,
    "MobileNetV2": mobilenet_history,
    "NASNetMobile": nasnet_history,
    "VGG16": vgg16_history
}

plt.figure(figsize=(15, 6))

# --- Plot Validation Accuracy ---
plt.subplot(1, 2, 1)
for name, history in all_histories.items():
    plt.plot(history.history['val_accuracy'], label=name)
plt.title('Validation Accuracy Comparison')
plt.ylabel('Validation Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.grid(True)

# --- Plot Validation Loss ---
plt.subplot(1, 2, 2)
for name, history in all_histories.items():
    plt.plot(history.history['val_loss'], label=name)
plt.title('Validation Loss Comparison')
plt.ylabel('Validation Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Kumpulkan semua model object ke dalam satu dictionary
# GANTI DENGAN NAMA MODEL OBJECT ANDA
all_models = {
    "ResNet50": resnet_model,
    "EfficientNetB0": efficientnet_model,
    "DenseNet121": densenet_model,
    "MobileNetV2": mobilenet_model,
    "NASNetMobile": nasnet_model,
    "VGG16": vgg16_model
}

# --- 1. Ambil True Labels (Label Sebenarnya) ---
# Mengatur ulang generator untuk memastikan urutan data yang benar
test_generator.reset()
true_labels = test_generator.classes
class_names = list(test_generator.class_indices.keys())
print(f"Nama Kelas: {class_names}")

# --- 2. Fungsi untuk Membuat Confusion Matrix ---
def plot_confusion_matrix(model, model_name, generator, true_labels, class_names):

    # 1. Prediksi Probabilitas
    y_pred_proba = model.predict(generator)

    # 2. Konversi Probabilitas ke Label Biner (0 atau 1)
    # Karena ini klasifikasi biner dengan sigmoid, kita gunakan threshold 0.5
    y_pred_binary = (y_pred_proba > 0.5).astype("int32")

    # 3. Hitung Confusion Matrix
    cm = confusion_matrix(true_labels, y_pred_binary)

    # 4. Visualisasi
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix: {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    # 5. Tampilkan Classification Report (Precision, Recall, F1-Score)
    print(f"\nClassification Report for {model_name}:")
    print(classification_report(true_labels, y_pred_binary, target_names=class_names))

# --- 3. Eksekusi untuk Setiap Model ---
for name, model in all_models.items():
    print(f"\n===========================================")
    print(f"Menganalisis Kinerja: {name}")

    # Reset generator sebelum prediksi untuk memastikan urutan data
    test_generator.reset()

    # Plot dan Report
    plot_confusion_matrix(model, name, test_generator, true_labels, class_names)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import time

# Asumsi: all_models dan test_generator sudah didefinisikan dari langkah sebelumnya

# Kumpulkan semua history object dan model object
all_histories = {
    "ResNet50": resnet_history,
    "EfficientNet": efficientnet_history, # Dianggap EfficientNetB0
    "DenseNet": densenet_history, # Dianggap DenseNet121
    "MobileNetV2": mobilenet_history,
    "NASNetMobile": nasnet_history,
    "VGG16": vgg16_history
}
all_models = {
    "ResNet50": resnet_model,
    "EfficientNet": efficientnet_model,
    "DenseNet": densenet_model,
    "MobileNetV2": mobilenet_model,
    "NASNetMobile": nasnet_model,
    "VGG16": vgg16_model
}

# Inisialisasi daftar untuk menampung data
data_rows = []
NUM_INFERENCE_TESTS = 100 # Jumlah gambar yang akan diuji untuk Inference Speed
input_shape = IMG_SIZE + (3,)

# --- Loop Melalui Setiap Model ---
for name, model in all_models.items():
    print(f"Memproses metrik untuk: {name}...")

    # A. Prediksi dan Laporan Klasifikasi
    test_generator.reset()
    true_labels = test_generator.classes

    # Dapatkan prediksi probabilitas
    y_pred_proba = model.predict(test_generator)
    y_pred_binary = (y_pred_proba > 0.5).astype("int32")

    # Hitung Classification Report
    report = classification_report(true_labels, y_pred_binary, output_dict=True, zero_division=0)

    # Karena ini biner, kita bisa gunakan rata-rata 'weighted avg' dari report
    accuracy = report['accuracy']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']

    # B. Waktu Pelatihan (Training Time) dan Epoch

    # Early Stopping menghentikan training, jadi kita ambil epoch terakhir yang sebenarnya
    epochs_used = len(all_histories[name].history['loss'])
    # Colab tidak memberikan waktu per epoch, jadi kita akan hitung total waktu
    # NOTE: Anda harus MENCATAT MANUAL total waktu training dari Colab Output!
    # Di sini, kita akan menggunakan nilai placeholder yang realistis:

    # Placeholder Waktu (GANTI dengan data manual Anda)
    if name == 'MobileNetV2':
        training_time = "5m:10s"
    elif name == 'EfficientNet':
        training_time = "6m:40s"
    elif name == 'ResNet50':
        training_time = "8m:50s"
    elif name == 'VGG16':
        training_time = "10m:20s"
    else:
        training_time = f"{epochs_used} E" # Placeholder jika waktu tidak tersedia

    # C. Inference Speed (Waktu Prediksi)

    # Buat data dummy untuk inferensi
    dummy_input = np.random.rand(NUM_INFERENCE_TESTS, input_shape[0], input_shape[1], input_shape[2])

    start_time = time.time()
    # Lakukan prediksi pada data dummy untuk mendapatkan waktu yang akurat
    model.predict(dummy_input, verbose=0)
    end_time = time.time()

    # Kecepatan Inferensi: Waktu per gambar (detik/gambar)
    inference_speed = (end_time - start_time) / NUM_INFERENCE_TESTS

    # Tambahkan baris data
    data_rows.append({
        'Model': name,
        'Accuracy': f"{accuracy:.4f}",
        'Precision': f"{precision:.4f}",
        'Recall': f"{recall:.4f}",
        'F1-score': f"{f1_score:.4f}",
        'Epoch': epochs_used,
        'Training Time (Manual)': training_time,
        'Inference Speed (s/img)': f"{inference_speed:.4f}"
    })

# Buat DataFrame
df_comparison = pd.DataFrame(data_rows)
df_comparison = df_comparison.sort_values(by='Accuracy', ascending=False)

In [ ]:
print("\n=========================================================================")
print("Tabel 1. Perbandingan Metrik Kinerja Model Klasifikasi Sampah")
print("=========================================================================")
print(df_comparison.to_markdown(index=False, numalign="left", stralign="left"))